<a href="https://colab.research.google.com/github/Bjorn455215/B103021055/blob/main/hw02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW2 Web Crawler

# 摘要

本專案透過 Python 中的 requests 與 BeautifulSoup 套件，對博客來網站進行靜態網頁爬蟲，針對博客來網站內擷取搜尋結果中之書名、作者、價格、商品連結等資訊，最終資料以 CSV 檔格式儲存。

# 引言

隨著網路資料日益豐富，能夠有效爬取網頁並擷取所需資訊成為資料分析與商業應用的重要技能。博客來作為台灣具代表性的線上書店，擁有大量書籍資料與穩定的網頁架構，適合作為靜態網頁爬蟲練習對象，本實作選定博客來網站作為研究主題，進一步探討如何運用 requests、BeautifulSoup 等工具，對目標頁面進行資料擷取，並彙整成一份可供後續應用的資料集。

# 方法

1.搜尋關鍵字設定：選擇「財經」作為博客來搜尋主題。

2.程式語言與工具：

* Python

* 套件：requests、BeautifulSoup、pandas、urllib.parse

* 平台：Google Colab

3.爬取流程：

* 使用 requests 模組對博客來搜尋結果進行 HTML 請求。

* 利用 BeautifulSoup 解析網頁結構，擷取每本書的書名、作者、價格、圖片等。

* 整理為表格資料並儲存成 CSV。

**1.博客來網站暢銷物品(不限書籍)資料爬取**

In [ ]:
#引入模組
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 設定目標網址與 headers（模擬瀏覽器）
url = 'https://www.books.com.tw/web/sys_hourstop/home'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0'}
res = requests.get(url, headers=headers)
soup = BeautifulSoup(res.text, 'lxml')

# 初始化空的結果清單
data = []

#抓取商品區塊
product1 = soup.find('div', class_='container_24 main_wrap clearfix')
product = product1.find_all('li', class_='item')

#對每個商品區塊逐一擷取資訊
for i in product:
    # 擷取排名
    rank1 = i.find('div', class_='stitle').find('p', class_='no_list')
    rank = rank1.find('strong', class_='no').text if rank1 and rank1.find('strong', class_='no') else '無排名'
    # 擷取圖片網址
    img1 = i.find('a').find('img', class_='cover')
    img = img1['src'] if img1 else '無圖片'
    # 商品區塊
    div = i.find('div', class_='type02_bd-a')
    if not div:
        continue #沒商品就跳過
    # 商品連結、名稱
    name1 = div.find('h4').find('a')
    link_product = name1['href'] if name1 else '無連結'
    name = name1.text.strip() if name1 else '無商品名稱'
    # 價格、作者、作者連結
    price1 = div.find('ul', class_='msg')
    author = '無作者'
    link_author = '無作者相關商品連結'
    price = '無價格'
    if price1:
        li_tags = price1.find_all('li')
        # 抓第一個li裡的作者資訊
        if len(li_tags) > 0:
            author_tag = li_tags[0].find('a')
            if author_tag:
                author = author_tag.text.strip()
                link_author = author_tag.get('href', '無作者相關商品連結')
    if price1:
        # 價格（處理有折扣或單價情況）
        price_tag = price1.find('li', class_='price_a')
        if price_tag:
            strongs = price_tag.find_all('strong')
            if strongs:
                if len(strongs) == 2:  # 折扣與金額
                    discount = strongs[0].text.strip()
                    amount = strongs[1].text.strip()
                    price = f'{discount}折 {amount}元'
                else:
                    price = ''.join(s.text.strip() for s in strongs)
                    price = f'{price}元'
    # 組合成字典加入清單
    data.append({
        '排名': rank,
        '商品名稱': name,
        '圖片網址': img,
        '作者': author,
        '價格': price,
        '商品連結': link_product,
        '作者相關商品連結': link_author
    })
# 儲存為 CSV
df = pd.DataFrame(data)
df.to_csv('博客來全站熱銷榜.csv', index=False, encoding='utf-8-sig')

2.**書籍種類關鍵字搜尋功能**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse

#使用者輸入
keyword = input("請輸入想搜尋的關鍵字：")
max_pages = int(input("請輸入最多想爬取的頁數（建議不要超過10）："))

#網址與標頭設定
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                  '(KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0'
}
encoded_keyword = urllib.parse.quote(keyword) #把 中文關鍵字 自動轉換網址合法格式
base_url = "https://search.books.com.tw/search/query/cat/all/sort/1/v/1/spell/3/ms2/ms2_1/page"

#爬蟲主程式
results = []
for page in range(1, max_pages + 1):
    print(f" 正在爬取第 {page} 頁 ...")
    url = f"{base_url}/{page}/key/{encoded_keyword}"
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")

    # 最外層容器
    container = soup.find('div', class_='table-searchbox clearfix')
    if not container:
        print(" 找不到主要內容，可能頁面結構改變。")
        continue

    # 每一個商品項目
    books = container.find_all('div', class_='table-td')
    if not books:
        print(f" 第 {page} 頁沒有商品資料")
        continue

    for book in books:
        # 書名
        name_tag = book.find('h4')
        name = name_tag.get_text(strip=True) if name_tag else '無書名'

        # 商品連結
        link_tag = name_tag.find('a') if name_tag else None
        link = "https:" + link_tag['href'] if link_tag and 'href' in link_tag.attrs else '無連結'

        # 若無書名或連結就跳過
        if not name or not link:
          continue
        # 作者
        author_tag = book.find('p', class_='author')
        author = author_tag.get_text(strip=True).replace("\xa0", "").replace("中文書", "") if author_tag else "無作者"

        # 價格
        price_ul = book.find('ul', class_='price clearfix')
        price = price_ul.get_text(strip=True).replace("\xa0", "").replace("優惠價：", "").replace("元", " 元") if price_ul else '無價格'


        results.append({
            '書名': name,
            '作者': author,
            '價格': price,
            '商品連結': link,
        })

# === 存成 CSV ===
if results:
    df = pd.DataFrame(results)
    output_file = f"博客來搜尋_{keyword}_top{len(results)}.csv"
    df.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"\n 已成功儲存 {len(results)} 筆資料到：{output_file}")
else:
    print(" 沒有抓到任何資料。")


請輸入想搜尋的關鍵字：人生
請輸入最多想爬取的頁數（建議不要超過10）：1
 正在爬取第 1 頁 ...

 已成功儲存 120 筆資料到：博客來搜尋_人生_top120.csv


試搜尋關鍵字"人生" 且 頁數爬取輸入1頁後，發現會產生.csv中會有無作者、無連結、無價錢等狀況被抓進去。

於是新增刪除這些情況的程式碼。

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse
import time

keyword = input("請輸入想搜尋的關鍵字：")
max_pages = int(input("請輸入最多想爬取的頁數（建議不要超過10）："))
encoded_keyword = urllib.parse.quote(keyword)

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                  '(KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0'
}

results = []

for p in range(1, max_pages + 1):
    url = f"https://search.books.com.tw/search/query/cat/all/sort/1/v/1/spell/3/ms2/ms2_1/page/{p}/key/{encoded_keyword}"
    print(f" 正在爬取第 {p} 頁：{url}")

    try:
        res = requests.get(url, headers=headers)
        res.encoding = 'utf-8'
        soup = BeautifulSoup(res.text, "lxml")

        container = soup.find('div', class_='table-searchbox clearfix')
        if not container:
            print(" 無搜尋結果或找不到區塊")
            break

        items = container.find_all('div', class_='table-td')
        if not items:
            print(" 無商品項目")
            break

        for item in items:
            # 書名與連結
            title_tag = item.find('h4')
            if not title_tag or not title_tag.find('a'):
                continue
            title = title_tag.find('a').text.strip()
            rel_link = title_tag.find('a')['href'].strip()
            link = "https:" + rel_link if rel_link.startswith("//") else rel_link

            # 作者
            author_tag = item.find('p', class_='author')
            author = author_tag.text.strip() if author_tag else "無作者"

            # 價格
            price_block = item.find('ul', class_='price clearfix')
            price = price_block.text.strip().replace('\n', '') if price_block else "無價格"

            # 圖片網址
            box_div = item.find('div', class_='box')
            if box_div:  # 如果找到了 div.box
              img_tag = box_div.find('img')
              if img_tag:  # 確保 img 標籤存在
                img_link = img_tag.get('data-src') if img_tag else "無圖片"
              else:
                img_link = "無圖片"
            else:
              img_link = "無圖片"


            if title != '無書名' and author != '無作者' and price != '無價格' and link != '無連結':
                results.append({
                    '書名': title,
                    '作者': author,
                    '價格': price,
                    '商品連結': link,
                    '圖片連結': img_link
                })

        time.sleep(2)  # 加等待，避免被擋

    except Exception as e:
        print(f" 第 {p} 頁錯誤：{e}")
        continue

# 儲存與顯示
df = pd.DataFrame(results)
output_name = f"博客來搜尋_{keyword}_top{len(df)}.csv"
df.to_csv(output_name, index=False, encoding='utf-8-sig')
print(f"\n 成功儲存 {len(df)} 筆資料到：{output_name}")


請輸入想搜尋的關鍵字：人生
請輸入最多想爬取的頁數（建議不要超過10）：1
 正在爬取第 1 頁：https://search.books.com.tw/search/query/cat/all/sort/1/v/1/spell/3/ms2/ms2_1/page/1/key/%E4%BA%BA%E7%94%9F

 成功儲存 60 筆資料到：博客來搜尋_人生_top60.csv


# 結果

**博客來網站暢銷物品資料爬取結果印出:**

In [ ]:
data #印出陣列後確實有抓到資料

[{'排名': '1',
  '商品名稱': '港仔茶里吃爆台灣！：逮丸呷透透！全部都「好好味」呀～【作者親簽＋限量贈品明信片】',
  '圖片網址': 'https://im1.book.com.tw/image/getImage?i=https://www.books.com.tw/img/001/101/99/0011019984.jpg&v=6808c131k&w=150&h=150',
  '作者': '茶里',
  '價格': '79折 331元',
  '商品連結': 'https://www.books.com.tw/products/0011019984?loc=P_0004_001',
  '作者相關商品連結': 'https://search.books.com.tw/search/query/key/%E8%8C%B6%E9%87%8C'},
 {'排名': '2',
  '商品名稱': '精準營養與肺癌治療',
  '圖片網址': 'https://im1.book.com.tw/image/getImage?i=https://www.books.com.tw/img/001/101/96/0011019668.jpg&v=68061e2ak&w=150&h=150',
  '作者': '陳晉興,許瑞芬,葉宜玲',
  '價格': '79折 474元',
  '商品連結': 'https://www.books.com.tw/products/0011019668?loc=P_0004_002',
  '作者相關商品連結': 'https://search.books.com.tw/search/query/key/%E9%99%B3%E6%99%89%E8%88%88%2C%E8%A8%B1%E7%91%9E%E8%8A%AC%2C%E8%91%89%E5%AE%9C%E7%8E%B2'},
 {'排名': '3',
  '商品名稱': '【MUJI 無印良品】黃麻購物袋A6',
  '圖片網址': 'https://im2.book.com.tw/image/getImage?i=https://www.books.com.tw/img/N00/179/40/N001794049.jpg&v=67a963bck&w=150&h=150

**書籍種類關鍵字搜尋之結果印出**

In [ ]:
results #印出陣列後確實有抓到資料

[{'書名': '世界盡頭的咖啡館：這一生，我為何而存在?(全球每19秒售出1本!療癒千萬人的暢銷經典，定位人生的神奇之書)',
  '作者': '約翰‧史崔勒基\xa0\xa0Elsa',
  '價格': '優惠價: 79 折, 395 元',
  '商品連結': 'https://search.books.com.tw/redirect/move/key/%E4%BA%BA%E7%94%9F/area/mid/item/0011014199/page/1/idx/1/cat/001/pdf/1/spell/3',
  '圖片連結': 'https://im1.book.com.tw/image/getImage?i=https://www.books.com.tw/img/001/101/41/0011014199.jpg&w=187&h=187&v=67bda9bb'},
 {'書名': '人生就是一次次的得到與放下',
  '作者': '黃大米',
  '價格': '優惠價: 79 折, 292 元',
  '商品連結': 'https://search.books.com.tw/redirect/move/key/%E4%BA%BA%E7%94%9F/area/mid/item/0011015062/page/1/idx/2/cat/001/pdf/1/spell/3',
  '圖片連結': 'https://im1.book.com.tw/image/getImage?i=https://www.books.com.tw/img/001/101/50/0011015062.jpg&w=187&h=187&v=67b6232b'},
 {'書名': '隱性潛能：華頓商學院最具影響力教授，突破天賦極限的實證科學【博客來獨家版.附人生領航指南書】',
  '作者': '亞當．格蘭特\xa0\xa0洪慧芳',
  '價格': '優惠價: 79 折, 379 元',
  '商品連結': 'https://search.books.com.tw/redirect/move/key/%E4%BA%BA%E7%94%9F/area/mid/item/0010995697/page/1/idx/3/cat/001/pdf/1/spell/3',
  '圖片連結': 'ht

# 問題與挑戰

* 技術挑戰
  * 部分網站（如 PChome、蝦皮、foodpanda）使用 AJAX 或 JavaScript 動態載入資料，無法直接透過 requests 或 BeautifulSoup 獲取，需進一步分析網路請求結構或使用 Selenium 等工具。

* 資料限制
 * 爬取結果中可能包含空值（如「無書名」、「無價格」），這些資料若未過濾，將影響後續分析的準確性與可讀性。

# 結論

* 成功建立爬蟲程式，實作輸入關鍵字即自動搜尋博客來網站，並擷取書籍名稱、價格、作者與連結等資訊。

* 程式具備資料過濾功能，能有效排除不完整資料，提高資料品質。

* 此功能可擴展應用至其他以靜態資料為主的平台，並搭配視覺化工具進行趨勢分析與推薦系統設計。

# 參考資料

[博客來官方網站](https://www.books.com.tw/)